In [5]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

#######딥러닝 라이브러리##########
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN

tf.keras.backend.set_floatx('float64')

2023-10-31 06:48:59.655606: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 06:48:59.655671: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 06:48:59.655697: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 06:48:59.663518: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 06:49:00.482915: W tensorflow/compiler/

In [6]:
# 맵플롯립 시각화 및 한국어 설정 (맷플롯립 시각화 시)
import matplotlib.pyplot as plt

# 브라우저에서 바로 그려지도록(그래프가 화면에 뜨지 않을 때)
%matplotlib inline

# 그래프(figure)에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정(나눔고딕체)
plt.rc('font', family='NanumGothic')

# 조용하 강사님이 올려주신 코드(한글화-matplotlib)
!pip install koreanize-matplotlib
import koreanize_matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [7]:
train=pd.read_csv('train.csv', encoding = 'cp949')
test=pd.read_csv('test.csv', encoding = 'cp949')
submission=pd.read_csv('sample_submission.csv', encoding = 'cp949')

In [8]:
# 컬럼명 변환을 위한 리스트 할당
train_col = ['num', 'date_time', 'power', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
test_col = ['num', 'date_time', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
train.columns=train_col
test.columns=test_col

In [9]:
def make_time(train):
    """
    시간 관련 변수를 추가하기 위한 함수
    """
    train['date_time'] = pd.to_datetime(train.date_time)
    
    train['month'] = train.date_time.dt.month                    # 월(숫자)
    train['day'] = train.date_time.dt.day                        # 일(숫자)
    train['hour'] = train.date_time.dt.hour                      # 시(숫자)
    train['weekday'] = train.date_time.dt.weekday                # 요일(숫자)
    train['dayofyear'] = train.date_time.dt.dayofyear            # 연 기준 몇일째(숫자)
    # train['week']=train.date_time.dt.weekofyear
    
    ## cyclical encoding
    train['sin_time'] = np.sin(2*np.pi*train.hour/24)
    train['cos_time'] = np.cos(2*np.pi*train.hour/24)
    
    ### 공휴일 변수 추가
    train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
    train.loc[('2020-08-17'<=train.date_time)&(train.date_time<'2020-08-18'), 'holiday'] = 1
    
    
    ### 날씨 변수 추가
    train['THI'] = 1.9*train['temp'] - 0.55*(1-train['hum'])*(1.8*train['temp']-26)+32                                # 불쾌지수
    train['chill']=13.12+0.6215*train['temp']-11.37*train['wind']**0.16+0.3965*train['wind']**0.16*train['temp']        # 체감온도
    
    train['냉방도시'] = 0  # 냉방이 필요하지 않은 도시로 초기화

    # 여름 동안 냉방이 필요한 도시로 설정
    train.loc[(train['temp'] > 30) & (train['hum'] > 70), '냉방도시'] = 1
    
    return train

In [10]:
def make_nan(test):
    
    # 결측치 처리
    test['wind']=test['wind'].interpolate(method='linear')
    test['rain']=test['rain'].interpolate(method='linear')
    test['sun']=test['sun'].interpolate(method='linear')
    
    test['temp']=test['temp'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    test['hum']=test['hum'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    
    test['cooler']=test['cooler'].fillna(0)
    test['solar']=test['solar'].fillna(0)
    
    return test

In [11]:
train = make_time(train)
#train = make_power(train)
test = make_nan(test)
test = make_time(test)

In [12]:
def make_weather(data):
    weather=data.groupby(['num','dayofyear'])['temp'].agg({'max','min','mean'}).reset_index()
    weather['temp_range']=weather['max']-weather['min']
    
    data=pd.merge(data,weather,on=['num','dayofyear'],how='left')
    return data

In [13]:
#train = make_power(train)
train = make_weather(train)
test=make_weather(test)

In [14]:
train=train[train['num']==1]
test=test[test['num']==1]

In [15]:
from sklearn.preprocessing import MinMaxScaler

# 입력 변수 선택
feature_cols = ['temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'holiday', 'THI', 'chill', '냉방도시', 'mean', 'min', 'max', 'temp_range','date_time']
## 종속변수 선택
label_cols = ['power']

In [16]:
feature_df = pd.DataFrame(train[feature_cols], columns=feature_cols).set_index('date_time')
label_df = pd.DataFrame(train[label_cols], columns=label_cols)

In [21]:
## 딥러닝 학습을 위해 numpy array로 변환
feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

In [24]:
feature_np.shape

(2040, 14)

In [19]:
label_df

,power
0,8179.056
1,8135.640
2,8107.128
3,8048.808
4,8043.624
...,...
2035,8714.952
2036,8740.224
2037,8730.504
2038,8725.968


In [23]:
np.random.rand(1000, 10, 1).shape

(1000, 10, 1)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed

# 가상의 시계열 데이터 생성
def generate_time_series(n_samples, n_timesteps, n_features, n_outputs):
    X = np.random.rand(n_samples, n_timesteps, n_features)
    y = np.random.rand(n_samples, n_outputs, n_features)
    return X, y

n_samples = label_df.shape[0]
n_timesteps = 168
n_features = 10
n_outputs = 168  # 예제에서는 다음 5개 스텝을 예측

X, y = generate_time_series(n_samples, n_timesteps, n_features, n_outputs)

# 모델 구축
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_timesteps, n_features)))
model.add(TimeDistributed(Dense(n_features))  # TimeDistributed 레이어를 사용하여 각 스텝에 대한 출력을 생성
model.compile(optimizer='adam', loss='mse')  # 회귀 문제이므로 평균 제곱 오차를 사용합니다.

# 모델 훈련
model.fit(X, y, epochs=10, batch_size=32)

# 새로운 데이터에 대한 예측
new_data = np.random.rand(1, n_timesteps, n_features)
prediction = model.predict(new_data)

print(f'Prediction: {prediction}')
